Import all required tools

In [1]:
import ixmp
import message_ix
import numpy as np
import pandas as pd
import yaml

from collections.abc import Mapping
from itertools import repeat
from message_ix.models import MESSAGE_ITEMS
from message_ix.utils import make_df

%matplotlib inline

mp = ixmp.Platform()

<IPython.core.display.Javascript object>

This part is just to generate a dummy scenario as basis for adding technology parameters. Need to be removed once integrated

In [2]:
model = "Westeros Electrified"

base = message_ix.Scenario(mp, model=model, scenario="baseline")
scen = base.clone(
    model,
    "add_tech",
    "introducing the add_tech feature on MESSAGEix",
    keep_solution=False,
)
scen.check_out()

year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df["year_vtg"], year_df["year_act"]
model_horizon = scen.set("year")
country = "Westeros"

#scen.commit(comment="Introducing emissions and setting an upper bound")
#scen.set_as_default()
#scen.solve()
#scen.var("OBJ")["lvl"]

Call all data required by add_tech module

In [3]:
with open('tech_data.yaml','r') as stream:
    tech_data = yaml.safe_load(stream)

# As if these parameters provided in the arguments
#regions = ['Westeros', 'Westerlands']
#modes = ['standard']
#emissions = ['CO2','CH4']
#time = ['year']

regions = []
modes = []
emissions = []
time = []
commodities = []
levels = []

first_active_year = tech_data['model_data'].get('first_active_year')
# If those are not provided, then this block of code is needed to retrieve them from the data input
if not regions:
    regions = list(tech_data.get('model_data',{}).get('DACCS',{}).get('fix_cost',{}).get('node_loc').keys())
if not modes:
    modes = list(tech_data.get('model_data',{}).get('DACCS',{}).get('var_cost',{}).get('mode').keys())
if not emissions:
    emissions = list(tech_data.get('model_data',{}).get('DACCS',{}).get('emission_factor',{}).get('emission').keys())
if not time:
    time = list(tech_data.get('model_data',{}).get('DACCS',{}).get('var_cost',{}).get('time').keys())
if not commodities:
    commodities = list(tech_data.get('model_data',{}).get('DACCS',{}).get('input',{}).get('commodity').keys())
if not levels:
    levels = list(tech_data.get('model_data',{}).get('DACCS',{}).get('input',{}).get('level').keys())
    
#list(scen.set('node'))

Define years vintage and active according to `first_active_year` and scenarios year

In [4]:
years_vtg_act = scen.vintage_and_active_years()
years_vtg_act = years_vtg_act[years_vtg_act['year_vtg'] >= first_active_year]

#years_vtg_act

In [5]:
parameters = {}
for tech in set(tech_data) - set(['model_data']):
    parameters.update({par: list(MESSAGE_ITEMS[par]['idx_names']) for par in set(tech_data[tech])})
data = {par: [] for par in list(parameters.keys())}

#list(scen.set('node'))

In [6]:
parameters

{'var_cost': ['node_loc',
  'technology',
  'year_vtg',
  'year_act',
  'mode',
  'time'],
 'input': ['node_loc',
  'technology',
  'year_vtg',
  'year_act',
  'mode',
  'node_origin',
  'commodity',
  'level',
  'time',
  'time_origin'],
 'growth_activity_lo': ['node_loc', 'technology', 'year_act', 'time'],
 'fix_cost': ['node_loc', 'technology', 'year_vtg', 'year_act'],
 'initial_new_capacity_up': ['node_loc', 'technology', 'year_vtg'],
 'emission_factor': ['node_loc',
  'technology',
  'year_vtg',
  'year_act',
  'mode',
  'emission'],
 'level_cost_activity_soft_up': ['node_loc', 'technology', 'year_act', 'time'],
 'construction_time': ['node_loc', 'technology', 'year_vtg'],
 'output': ['node_loc',
  'technology',
  'year_vtg',
  'year_act',
  'mode',
  'node_dest',
  'commodity',
  'level',
  'time',
  'time_dest'],
 'capacity_factor': ['node_loc', 'technology', 'year_vtg', 'year_act', 'time'],
 'level_cost_activity_soft_lo': ['node_loc', 'technology', 'year_act', 'time'],
 'growth

Creating basic dataframe to be filled in later

In [7]:
# Basic DataFrame
for tech, par_dict in tech_data.items():
    if tech != 'model_data':
        for par, par_data in par_dict.items():
            if not isinstance(par_data, Mapping):
                par_data = {'value': par_data, 'unit': '-'}
            # identify parameters by `year` dimensions
            # then add the year data as kwargs when creating basic dataframe
            if parameters[par] == ['node_loc', 'technology', 'year_vtg']:
                kwargs = {'year_vtg': sorted(list(set(years_vtg_act['year_vtg'])))}
            if parameters[par] == ['node_loc', 'technology', 'year_act']:
                kwargs = {'year_vtg': sorted(list(set(years_vtg_act['year_act'])))}
            elif all(e in ['node_loc', 'technology', 'year_vtg', 'year_act'] for e in parameters[par]):
                kwargs = {'year_vtg': years_vtg_act['year_vtg'],
                          'year_act': years_vtg_act['year_act']}
            data[par].append(
                make_df(
                    par,
                    technology=tech,
                    value=par_data['value'],
                    unit=par_data['unit'],
                    **kwargs,
                ))
            # adding additional 
            if 'emission' in parameters[par]:
                data[par] = data[par]*len(emissions)
                for e in range(len(emissions)):
                    data[par][e] = data[par][e].assign(emission=emissions[e])
            if 'mode' in parameters[par]:
                data[par] = data[par]*len(modes)
                for m in range(len(modes)):
                    for e in range(len(data[par])):
                        data[par][e] = data[par][e].assign(mode=modes[m])
            if 'time' in parameters[par]:
                data[par] = data[par]*len(time)
                for t in range(len(time)):
                    for e in range(len(data[par])):
                        data[par][e] = data[par][e].assign(time=time[t])
                        if 'time_origin' in parameters[par]:
                            data[par][e] = data[par][e].assign(time_origin=time[t])
                        if 'time_dest' in parameters[par]:
                            data[par][e] = data[par][e].assign(time_dest=time[t])
            if 'commodity' in parameters[par]:
                data[par] = data[par]*len(commodities)
                for c in range(len(commodities)):
                    for e in range(len(data[par])):
                        data[par][e] = data[par][e].assign(commodity=commodities[c])
            if 'level' in parameters[par]:
                data[par] = data[par]*len(levels)
                for l in range(len(levels)):
                    for e in range(len(data[par])):
                        data[par][e] = data[par][e].assign(level=levels[l])
            

data = {k: pd.concat(v).reset_index(drop=True) for k, v in data.items()}

In [8]:
data['level_cost_activity_soft_up']

,node_loc,technology,year_act,time,value,unit
0,None,DACCS,700,year,0.5,-
1,None,DACCS,710,year,0.5,-
2,None,DACCS,720,year,0.5,-
3,None,DACCS,710,year,0.5,-
4,None,DACCS,720,year,0.5,-
5,None,DACCS,720,year,0.5,-


In [9]:
# Expanded DataFrame
data_expand ={par: [] for par in list(parameters.keys())} 
for par in list(parameters.keys()):
    print(par)
    for tech, diffs in tech_data['model_data'].items():
        if tech != 'first_active_year':
            for reg in regions:
                multiplier = []
                for i in range(len(data[par])):
                    if 'year_vtg' in parameters[par]:
                        multiplier.append(
                            np.prod([diffs.get(par,{}).get('node_loc',{}).get(reg,1), # by region
                                     ((1+diffs.get(par,{}).get('year_vtg',{}).get('rate',0))
                                         **(data[par]['year_vtg'][i]-first_active_year)), # by year_vtg
                                     ((1+diffs.get(par,{}).get('year_act',{}).get('rate',0))
                                             **(data.get(par,{}).get('year_act',{}) # by year_act
                                                .get(i,data[par]['year_vtg'][i])-data[par]['year_vtg'][i])),
                                     diffs.get(par,{}).get('mode',{}).get(data.get(par,{}).get('mode',{}).get(i),1), # by mode
                                     diffs.get(par,{}).get('emission',{}).get(data.get(par,{}).get('emission',{}).get(i),1), # by emission
                            ])
                        )
                    else:
                        multiplier.append(
                            np.prod([diffs.get(par,{}).get('node_loc',{}).get(reg,1), # by region
                                     diffs.get(par,{}).get('mode',{}).get(data.get(par,{}).get('mode',{}).get(i),1), # by mode
                                     diffs.get(par,{}).get('emission',{}).get(data.get(par,{}).get('emission',{}).get(i),1), # by emission
                            ])
                        )


                value = data[par]['value']*multiplier
                # node origin is assumed to be always the same of the node
                if 'node_origin' in parameters[par]:
                    kwargs = {'node_origin': reg}
                else:
                    kwargs = {}
                # node destination is assumed to be always the same of the node
                if 'node_dest' in parameters[par]:
                    kwargs = {'node_dest': reg}
                else:
                    kwargs = {}
                # assigning data expansion
                data_expand[par].append(
                    data[par].assign(node_loc=reg,value=value, **kwargs)
                   )

var_cost
input
growth_activity_lo
fix_cost
initial_new_capacity_up
emission_factor
level_cost_activity_soft_up
construction_time
output
capacity_factor
level_cost_activity_soft_lo
growth_activity_up
technical_lifetime
growth_new_capacity_up
inv_cost
abs_cost_activity_soft_up


In [10]:
pd.concat(data_expand['inv_cost'])


,node_loc,technology,year_vtg,value,unit
0,Westeros,DACCS,700,2500.000000,$/kW
1,Westeros,DACCS,700,2500.000000,$/kW
2,Westeros,DACCS,700,2500.000000,$/kW
3,Westeros,DACCS,710,1496.842348,$/kW
4,Westeros,DACCS,710,1496.842348,$/kW
5,Westeros,DACCS,720,896.214806,$/kW


In [11]:
data_expand = {k: pd.concat(v) for k, v in data_expand.items() 
               if k in parameters.keys()}

In [12]:
with pd.ExcelWriter('printed_data.xlsx', engine='xlsxwriter', mode='w') as writer:
    for sheet_name, sheet_data in data_expand.items():
        sheet_data.to_excel(writer, sheet_name=sheet_name, index=False)